In [1]:
import os
import glob
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import functools as ft
import operator as op
from module import JP_Parser

In [2]:
os.chdir('/Users/sheg/Documents/projects/flashcards/japanese/data')

In [3]:
try:
    sent = pd.read_pickle('sent.pkl')
    vocab = pd.read_pickle('vocab.pkl')
    kanji = pd.read_pickle('kanji.pkl')
    jp_morpho = pd.read_pickle('jp_morpho.pkl')
except:
    pass

# Морфологический анализ

## Части речи

In [65]:
class TimeOutException(BaseException):
    pass

In [66]:
import timeout_decorator
import functools as ft

## Морфологический разбор корпуса

In [7]:
import datetime, time

In [8]:
def append_dump(df, store, key='df'):
    store.append(key, df,
                 data_columns=True,
                 min_itemsize={'word': 100,
                               'conjugated': 50,
                               'pos': 30,
                               'conjugated': 50,
                               'inflection': 50,
                               'base_form': 50,
                               'reading': 50
                              },
                 encoding='utf8',
                 format='t')

In [43]:
jp_morpho_h5 = pd.HDFStore('jp_morpho.h5')

In [57]:
good_indicies = []
jp_morpho = pd.DataFrame()

Если расчёт оборвался, можно импортировать индексы уже обработанных предложений

In [12]:
jp_morpho = jp_morpho_h5.get('df')
good_indicies = list( set( jp_morpho.index.get_level_values(0) ) )

In [44]:
jp_sent = sent[sent.lang == 'jp']

In [58]:
n_processed = len(good_indicies)

In [60]:
save_every = 500
save_str = ''
delta = 10

for row in jp_sent.itertuples():  

    start_time = time.perf_counter_ns()
    
    if (n_processed > 0) and (n_processed % save_every == 0):
        
        append_dump(jp_morpho, jp_morpho_h5, 'df')
        jp_morpho = pd.DataFrame()
    
    try:
        if row.Index in good_indicies:
            continue
        
        df = JP_Parser.process_item(row)
        jp_morpho = jp_morpho.append(df)
        good_indicies.append(row.Index)        
        n_processed += 1
        proceed_str = '{:0.2f}% was proceed'.format( 100 * n_processed / len(jp_sent))
        
    except:
        pass

    print(len(print_str) * ' ', end='\r')
    print_str = proceed_str + save_str
    print(print_str, end='\r')

    end_time = time.perf_counter_ns()
    
    try:
        save_vs_performance[save_every] += (end_time - start_time) / save_every
    except KeyError:
        save_vs_performance[save_every] = 0
    
    if (n_processed > 0) and (n_processed % save_every == save_every - 1):
        effective_save_every = min(save_vs_performance, key=save_vs_performance.get)

        save_every = effective_save_every
        
        save_every += np.random.randint(-delta, delta)
        if save_every <= 0:
                save_every += 2 * delta
                
        save_str = f', save every = {save_every}'
    
else:
    append_dump(jp_morpho, jp_morpho_h5, 'df')
    jp_morpho = pd.DataFrame()

In [110]:
jp_morpho = jp_morpho_h5.get('df')

Дообработаем то, что не обработалось

In [123]:
unproceed_ids = set(jp_sent.index) - set(jp_morpho.index.get_level_values(0) )
unproceed_ids = list(unproceed_ids)

In [124]:
sent.loc[unproceed_ids]

,lang,text,user,links,rating,audio


In [114]:
bad_sentence_ids = [7956559]

In [116]:
for bad in bad_sentence_ids:
    try:
        unproceed_ids.remove(bad)
    except:
        pass

In [117]:
for row in sent.loc[unproceed_ids].itertuples():
    df = JP_Parser.process_item(row)
    jp_morpho = jp_morpho.append(df)

In [118]:
len( jp_morpho.index.get_level_values(0).unique() )

182309

In [119]:
len( sent[sent.lang == 'jp'] )

182309

In [120]:
sent.drop(bad_sentence_ids, inplace=True)

In [121]:
jp_morpho.to_pickle('jp_morpho.pkl')